In [1]:
def as_list(x):
    if type(x) is list:
        return x
    else:
        return [x]

In [2]:
from riotwatcher import LolWatcher, ApiError
import pandas as pd
import numpy as np

# global variables
api_key = 'RGAPI-aa3b849d-42de-48ba-a0df-58a1d386c5b1'
watcher = LolWatcher(api_key)
my_region = 'euw1'


In [3]:
# fetch matchlist by puuid
me = watcher.summoner.by_name(my_region, 'Kelody')
# me = watcher.summoner.by_name(my_region, 'daimox')
my_matches = watcher.match.matchlist_by_puuid("EUROPE", me['puuid'])
matches = []
matchTimeLine = []
participants = []


In [4]:
# fetch game, participant and match timeline and transform into dataframe
for matchId in my_matches:
    match_detail = watcher.match.by_id("EUROPE", matchId)
    timeline = watcher.match.timeline_by_match('EUROPE', matchId)

    match = {}
    match['matchId'] = matchId
    match['gameDuration'] = match_detail['info']['gameDuration']
    match['gameType'] = match_detail['info']['gameType']
    match['gameVersion'] = match_detail['info']['gameVersion']
    matches.append(match)

    for index, row in enumerate(match_detail['info']['participants']):
        participants_row = {}
        participants_row['matchId'] = matchId
        participants_row['participant'] = index + 1
        participants_row['summonerName'] = row['summonerName']
        participants_row['teamPosition'] = row['teamPosition']
        participants_row['championName'] = row['championName']
        participants_row['kda'] = f"{row['kills']}/{row['deaths']}/{row['assists']}"
        participants_row['champLevel'] = row['champLevel']
        participants_row['totalDamageToChampions'] = row['totalDamageDealtToChampions']
        participants_row['totalDamageTaken'] = row['totalDamageTaken']
        participants_row['totalHeal'] = row['totalHeal']
        participants_row['totalMinionsKilled'] = row['totalMinionsKilled']
        participants_row['goldEarned'] = row['goldEarned']
        participants_row['summoner1'] = row['summoner1Id']
        participants_row['summoner2'] = row['summoner2Id']
        participants_row['item0'] = row['item0']
        participants_row['item1'] = row['item1']
        participants_row['item2'] = row['item2']
        participants_row['item3'] = row['item3']
        participants_row['item4'] = row['item4']
        participants_row['item5'] = row['item5']
        participants_row['item6'] = row['item6']
        participants_row['win'] = row['win']
        participants.append(participants_row)

    for row in timeline['info']['frames']:
        # frameCounter = 1
        # frame = row["{:02d}".format(frameCounter)]
        # frameCounter += 1
        for frameRow in row['events']:
            timeline_row = {}
            timeline_row['matchId'] = matchId
            eventCounter = 0
            # eventFrame = frameRow["{:02d}".format(eventCounter)]
            eventFrame = frameRow
            eventCounter += 1

            if eventFrame['type'] == 'CHAMPION_KILL':
                timeline_row['type'] = eventFrame['type']
                timeline_row['killerId'] = eventFrame['killerId']
                timeline_row['victimId'] = eventFrame['victimId']
                if 'assistingParticipantIds' in eventFrame:
                    timeline_row['assistingParticipantIds'] = eventFrame['assistingParticipantIds']
                if 'bounty' in eventFrame:
                    timeline_row['bounty'] = eventFrame['bounty']
                timeline_row['positionX'] = eventFrame['position']['x']
                timeline_row['positionY'] = eventFrame['position']['y']
                timeline_row['timestamp'] = eventFrame['timestamp']
                matchTimeLine.append(timeline_row)
            elif eventFrame['type'] == 'BUILDING_KILL':
                timeline_row['type'] = eventFrame['type']
                timeline_row['killerId'] = eventFrame['killerId']
                timeline_row['teamId'] = eventFrame['teamId']
                timeline_row['buildingType'] = eventFrame['buildingType']
                timeline_row['laneType'] = eventFrame['laneType']
                timeline_row['positionX'] = eventFrame['position']['x']
                timeline_row['positionY'] = eventFrame['position']['y']
                timeline_row['timestamp'] = eventFrame['timestamp']
                matchTimeLine.append(timeline_row)
            elif eventFrame['type'] == 'ITEM_PURCHASED':
                timeline_row['type'] = eventFrame['type']
                timeline_row['participantId'] = eventFrame['participantId']
                timeline_row['itemId'] = eventFrame['itemId']
                timeline_row['timestamp'] = eventFrame['timestamp']
                matchTimeLine.append(timeline_row)
            elif eventFrame['type'] == 'WARD_PLACED':
                timeline_row['type'] = eventFrame['type']
                timeline_row['wardType'] = eventFrame['wardType']
                timeline_row['creatorId'] = eventFrame['creatorId']
                timeline_row['timestamp'] = eventFrame['timestamp']
                matchTimeLine.append(timeline_row)
            elif eventFrame['type'] == 'WARD_KILL':
                timeline_row['type'] = eventFrame['type']
                timeline_row['wardType'] = eventFrame['wardType']
                timeline_row['killerId'] = eventFrame['killerId']
                timeline_row['timestamp'] = eventFrame['timestamp']
                matchTimeLine.append(timeline_row)
            elif eventFrame['type'] == 'LEVEL_UP':
                timeline_row['type'] = eventFrame['type']
                timeline_row['participantId'] = eventFrame['participantId']
                timeline_row['level'] = eventFrame['level']
                timeline_row['timestamp'] = eventFrame['timestamp']
                matchTimeLine.append(timeline_row)
            elif eventFrame['type'] == 'ELITE_MONSTER_KILL':
                timeline_row['type'] = eventFrame['type']
                timeline_row['killerId'] = eventFrame['killerId']
                timeline_row['teamId'] = eventFrame['killerTeamId']
                if 'assistingParticipantIds' in eventFrame:
                    timeline_row['assistingParticipantIds'] = eventFrame['assistingParticipantIds']
                timeline_row['positionX'] = eventFrame['position']['x']
                timeline_row['positionY'] = eventFrame['position']['y']
                timeline_row['timestamp'] = eventFrame['timestamp']
                timeline_row['monsterType'] = eventFrame['monsterType']
                matchTimeLine.append(timeline_row)


participantsDf = pd.DataFrame(participants)
matchesDf = pd.DataFrame(matches)
matchTimelineDf = pd.DataFrame(matchTimeLine)
matchTimelineDf["timestampMIN"] = round(matchTimelineDf["timestamp"] / 60000, 0);
print(participantsDf)
print(matchesDf)
print(matchTimelineDf)


             matchId  participant      summonerName teamPosition championName  \
0    EUW1_5715214240            1      dvärg pannan                      Yone   
1    EUW1_5715214240            2       Choupinator                     Akali   
2    EUW1_5715214240            3      Haferschloim                      Sona   
3    EUW1_5715214240            4     Th3Bourbonkid                       Lux   
4    EUW1_5715214240            5           Chuckee                    RekSai   
..               ...          ...               ...          ...          ...   
195  EUW1_5698842897            6      pipigangster               TwistedFate   
196  EUW1_5698842897            7            Kelody                      Sona   
197  EUW1_5698842897            8               rzα                   Sejuani   
198  EUW1_5698842897            9  Sprungweizen3000                   Nidalee   
199  EUW1_5698842897           10             Kinyy                      Pyke   

          kda  champLevel  

In [5]:
kelody = participantsDf[participantsDf["summonerName"] == "Kelody"][["matchId", "participant"]]
kelody

,matchId,participant
6,EUW1_5715214240,7
11,EUW1_5715130097,2
27,EUW1_5715125439,8
37,EUW1_5714224804,8
41,EUW1_5714130302,2
56,EUW1_5713855030,7
66,EUW1_5713459898,7
76,EUW1_5713455715,7
82,EUW1_5713340030,3
92,EUW1_5713375914,3


In [6]:
kelodyEvents = []

# (matchTimelineDf["assistingParticipantIds"] == kelody["participant"].values[0]
for index, match in kelody.iterrows():
    game = []
    assist = matchTimelineDf[(matchTimelineDf["matchId"] == match["matchId"]) & matchTimelineDf["assistingParticipantIds"].notna() & (matchTimelineDf["creatorId"] != match["participant"]) & (matchTimelineDf["killerId"] != match["participant"]) & (matchTimelineDf["victimId"] != match["participant"])]
    participantList = as_list(match["participant"])
    game = matchTimelineDf.loc[((matchTimelineDf["matchId"] == match["matchId"]) & ((matchTimelineDf["participantId"] == (match["participant"])) | 
    (matchTimelineDf["creatorId"] == match["participant"]) |
    (matchTimelineDf["killerId"] == match["participant"]) |
    (matchTimelineDf["victimId"] == match["participant"]) |
    (assist["assistingParticipantIds"].apply(lambda x: any(i in x for i in participantList)))))]
    kelodyEvents.append(game)
kelodyEventsDf = pd.concat(kelodyEvents, ignore_index=True)
kelodyEventsDf

,matchId,type,participantId,level,timestamp,itemId,killerId,victimId,assistingParticipantIds,bounty,positionX,positionY,teamId,buildingType,laneType,wardType,creatorId,monsterType,timestampMIN
0,EUW1_5715214240,LEVEL_UP,7.0,2.0,509,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
1,EUW1_5715214240,LEVEL_UP,7.0,3.0,509,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
2,EUW1_5715214240,ITEM_PURCHASED,7.0,NaN,11705,3802.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
3,EUW1_5715214240,CHAMPION_KILL,NaN,NaN,32747,NaN,9.0,3.0,"[7, 8]",270.0,6379.0,6532.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0
4,EUW1_5715214240,CHAMPION_KILL,NaN,NaN,46457,NaN,4.0,7.0,"[1, 2, 3, 5]",180.0,6329.0,6744.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1469,EUW1_5698842897,CHAMPION_KILL,NaN,NaN,1318330,NaN,8.0,3.0,"[6, 7, 9, 10]",163.0,3554.0,1961.0,NaN,NaN,NaN,NaN,NaN,NaN,22.0
1470,EUW1_5698842897,CHAMPION_KILL,NaN,NaN,1322801,NaN,10.0,1.0,"[6, 7, 8, 9]",163.0,4040.0,2507.0,NaN,NaN,NaN,NaN,NaN,NaN,22.0
1471,EUW1_5698842897,CHAMPION_KILL,NaN,NaN,1333026,NaN,9.0,5.0,"[6, 7, 8]",163.0,1277.0,1524.0,NaN,NaN,NaN,NaN,NaN,NaN,22.0
1472,EUW1_5698842897,CHAMPION_KILL,NaN,NaN,1342484,NaN,6.0,4.0,"[7, 8, 9]",180.0,2655.0,1789.0,NaN,NaN,NaN,NaN,NaN,NaN,22.0


Check how often a Player died in Early Game (pre 10 mins)

In [7]:
sumOfDeaths = []
for index, match in kelody.iterrows():
     deaths = len(kelodyEventsDf[(kelodyEventsDf["matchId"] == match["matchId"]) & (kelodyEventsDf["victimId"] == match["participant"]) & (kelodyEventsDf["timestamp"] <= 600000)].index)
     sumOfDeaths.insert(len(sumOfDeaths), deaths)
kelody["deathsPreMin10"] = sumOfDeaths
kelody

,matchId,participant,deathsPreMin10
6,EUW1_5715214240,7,6
11,EUW1_5715130097,2,4
27,EUW1_5715125439,8,5
37,EUW1_5714224804,8,1
41,EUW1_5714130302,2,2
56,EUW1_5713855030,7,0
66,EUW1_5713459898,7,2
76,EUW1_5713455715,7,0
82,EUW1_5713340030,3,3
92,EUW1_5713375914,3,5


Features:
Died from ganks in laning phase pre 15 min (check ID from enemy Jungler)

Died with ... Gold

Died from 1v1s on lane

Kills from 1v1s on lane

Good roaming (assists pre 15 min)

Warding (check ward placements pre 15 min) maybe in combination with ganks

CS per min (if data available)

Tower Kills/Assists

Objective Kills/Assists (Baron, Dragon, Herald)



Features on the tableau visualization:

Timeline

Champion Icons

In [8]:
print(timeline['info']['frames'][1])
participantsDf.to_json('participants.json', orient='records')
matchesDf.to_json('matches.json', orient='records')
kelodyEventsDf.to_json('kelodyEvents.json', orient='records')
kelody.to_json('kelodyStats.json', orient='records')
matchTimelineDf.to_json('matchTimeline.json', orient='records')


{'events': [{'level': 2, 'participantId': 1, 'timestamp': 513, 'type': 'LEVEL_UP'}, {'level': 3, 'participantId': 1, 'timestamp': 513, 'type': 'LEVEL_UP'}, {'level': 2, 'participantId': 2, 'timestamp': 513, 'type': 'LEVEL_UP'}, {'level': 3, 'participantId': 2, 'timestamp': 513, 'type': 'LEVEL_UP'}, {'level': 2, 'participantId': 3, 'timestamp': 513, 'type': 'LEVEL_UP'}, {'level': 3, 'participantId': 3, 'timestamp': 513, 'type': 'LEVEL_UP'}, {'level': 2, 'participantId': 4, 'timestamp': 513, 'type': 'LEVEL_UP'}, {'level': 3, 'participantId': 4, 'timestamp': 513, 'type': 'LEVEL_UP'}, {'level': 2, 'participantId': 5, 'timestamp': 513, 'type': 'LEVEL_UP'}, {'level': 3, 'participantId': 5, 'timestamp': 513, 'type': 'LEVEL_UP'}, {'level': 2, 'participantId': 6, 'timestamp': 513, 'type': 'LEVEL_UP'}, {'level': 3, 'participantId': 6, 'timestamp': 513, 'type': 'LEVEL_UP'}, {'level': 2, 'participantId': 7, 'timestamp': 513, 'type': 'LEVEL_UP'}, {'level': 3, 'participantId': 7, 'timestamp': 513, '